# Fortnight 1: Applied Exploration


### Demoed for: Conrad, Saul, Katja, Caitlyn

## Applied Exploration
### *from F1_2_DataEvaluation*

Go to the Hugging Face models page: https://huggingface.co/models
* Click `Text Classification`
* Find a different model and a dataset appropriate for testing it with than the ones we worked with today
    - many models will link to the datasets they were trained on, but you can find others at https://huggingface.co/datasets
    - write down some info about the models you found
        - what is it for?
        - who made it?
        - what kind of data was it trained on?
        - are they based on some other model and trained on new data (*fine-tuned*) for a specific task?
    - write down some info on the dataset you found
        - where did it come from?
        - how big is it?
        - what kind of labels does it classify?
* Evaluate the performance 
    - use some of the metrics we talked about today
    - describe in your own words how it performed
    

<div style="background: #007acc; color: white; border-radius: 10px; padding: 10px; width: fit-content; max-width: 70%; font-size: 1rem;">
    Dataset: "mteb/amazon_reviews_multi" <br/>
     - the data comes from amazon reviews <br/>
     - the English split has 200k rows of training data and 5k each for validation and test <br/>
     - it classifies to 1-5 stars <br/>
    Model: "philschmid/distilbert-base-multilingual-cased-sentiment-2" <br/>
     - predicts the likehood that the input is positive, negative, or neutral<br/>
     - it <br/>
     - <br/>
     - <br/>
</div>

In [5]:
!poetry add transformers datasets scikit-learn

The following packages are already present in the pyproject.toml and will be skipped:

  • transformers
  • datasets
  • scikit-learn

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [7]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_reviews_multi", "en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [9]:
print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


In [10]:
print(dataset["test"])

Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 5000
})


In [11]:
type(dataset["test"]["label"]) 


list

In [12]:
dataset["test"].features

{'id': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': Value(dtype='int32', id=None),
 'label_text': Value(dtype='string', id=None)}

In [13]:
# loop through features and print each type
for feature_name in dataset["test"].features:
    print(type(dataset["test"][feature_name]))


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [21]:
dataset["test"][:5]

{'id': ['en_0199937', 'en_0863335', 'en_0565010', 'en_0963290', 'en_0238156'],
 'text': ['Don’t waste your time!\n\nThese are AWFUL. They are see through, the fabric feels like tablecloth, and they fit like children’s clothing. Customer service did seem to be nice though, but I regret missing my return date for these. I wouldn’t even donate them because the quality is so poor.',
  'One Star\n\nI bought 4 and NONE of them worked. Yes I used new batteries!',
  "Totally useless\n\nOn first use it didn't heat up and now it doesn't work at all",
  "Gold filled earrings\n\nYou want an HONEST answer? I just returned from UPS where I returned the FARCE of an earring set to Amazon. It did NOT look like what I saw on Amazon. Only a baby would be able to wear the size of the earring. They were SO small. the size of a pin head I at first thought Amazon had forgotten to enclose them in the bag! I didn't bother to take them out of the bag and you can have them back. Will NEVER order another thing fr

In [22]:
display(dataset["test"][1000:1005]["label"])
display(dataset["test"][2000:2005]["label"])
display(dataset["test"][3000:3005]["label"])
display(dataset["test"][4000:4005]["label"])



[1, 1, 1, 1, 1]

[2, 2, 2, 2, 2]

[3, 3, 3, 3, 3]

[4, 4, 4, 4, 4]

In [24]:
display(dataset["test"][999:1005]["label"])
display(dataset["test"][1999:2005]["label"])
display(dataset["test"][2999:3005]["label"])
display(dataset["test"][3999:4005]["label"])



[0, 1, 1, 1, 1, 1]

[1, 2, 2, 2, 2, 2]

[2, 3, 3, 3, 3, 3]

[3, 4, 4, 4, 4, 4]

In [32]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

sample_size = 10

tokenizer = AutoTokenizer.from_pretrained("philschmid/distilbert-base-multilingual-cased-sentiment-2")
model = AutoModelForSequenceClassification.from_pretrained("philschmid/distilbert-base-multilingual-cased-sentiment-2")
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

results = classifier(dataset["test"]["text"][3999:4000+sample_size])
print(f"Here are the {len(results)} predictions")
# print(results)

total_correct = 0
for idx in range(sample_size):
    print(f"-------\nSample {idx+1} of {sample_size}:")
    print(dataset["test"]["text"][idx])


    actual_label_numeric = dataset["test"]["label"][idx]
    print(f"Stars: {actual_label_numeric}")

    if actual_label_numeric == 3:
        actual_label = "neutral"
    elif actual_label_numeric > 3:
        actual_label = "positive"
    else:
        actual_label = "negative"
    print(f"Actual label: {actual_label}")

    predicted_label = results[idx]["label"]
    print(f"Predicted label: {predicted_label}")

    if predicted_label == actual_label:
        total_correct += 1

print(f"-------\nAccuracy: {total_correct / sample_size}")

Here are the 11 predictions
-------
Sample 1 of 10:
Don’t waste your time!

These are AWFUL. They are see through, the fabric feels like tablecloth, and they fit like children’s clothing. Customer service did seem to be nice though, but I regret missing my return date for these. I wouldn’t even donate them because the quality is so poor.
Stars: 0
Actual label: negative
Predicted label: positive
-------
Sample 2 of 10:
One Star

I bought 4 and NONE of them worked. Yes I used new batteries!
Stars: 0
Actual label: negative
Predicted label: positive
-------
Sample 3 of 10:
Totally useless

On first use it didn't heat up and now it doesn't work at all
Stars: 0
Actual label: negative
Predicted label: positive
-------
Sample 4 of 10:
Gold filled earrings

You want an HONEST answer? I just returned from UPS where I returned the FARCE of an earring set to Amazon. It did NOT look like what I saw on Amazon. Only a baby would be able to wear the size of the earring. They were SO small. the size of

In [33]:
dataset["test"][3999]

{'id': 'en_0241214',
 'text': 'Easy setup with bluetooth to your phone\n\nEasy setup with bluetooth to your phone, and clear instructions provided. This is my second time buying this product, my first one had the USB cable snap off at the Y with almost a year of constant use, could have been my fault due to heavy use. Only complaint is that it is slightly quieter than max volume with a regular aux cord.',
 'label': 3,
 'label_text': '3'}

In [24]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

sample_size = 1000

tokenizer = AutoTokenizer.from_pretrained("philschmid/distilbert-base-multilingual-cased-sentiment-2")
model = AutoModelForSequenceClassification.from_pretrained("philschmid/distilbert-base-multilingual-cased-sentiment-2")
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

results = classifier(dataset["test"]["review_body"][0:sample_size])

total_correct = 0
for idx in range(sample_size):


    actual_label_numeric = dataset["test"]["stars"][idx]

    if actual_label_numeric == 3:
        actual_label = "neutral"
    elif actual_label_numeric > 3:
        actual_label = "positive"
    else:
        actual_label = "negative"

    predicted_label = results[idx]["label"]

    if predicted_label == actual_label:
        total_correct += 1

print(f"For {len(results)} samples, the accuracy is {total_correct / sample_size}")

For 1000 samples, the accuracy is 0.929


In [25]:
def calculate_accuracy(results, dataset):
    total_correct = 0
    sample_size = len(results)

    for idx in range(sample_size):
        actual_label_numeric = dataset["test"]["stars"][idx]

        if actual_label_numeric == 3:
            actual_label = "neutral"
        elif actual_label_numeric > 3:
            actual_label = "positive"
        else:
            actual_label = "negative"

        predicted_label = results[idx]["label"]

        if predicted_label == actual_label:
            total_correct += 1

    return total_correct / sample_size


In [27]:
calculate_accuracy(results, dataset)

0.929

In [28]:
sample_size = 1500
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 1500 samples, the accuracy is 0.862


In [29]:
sample_size = 2000
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 2000 samples, the accuracy is 0.837


In [30]:
sample_size = 2500
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 2500 samples, the accuracy is 0.74


In [33]:
# shuffle the dataset
dataset = dataset.shuffle()

In [35]:
sample_size = 2500
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 2500 samples, the accuracy is 0.7464


In [36]:
sample_size = 1000
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 1000 samples, the accuracy is 0.737


In [37]:
sample_size = 500
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 500 samples, the accuracy is 0.766


In [38]:
dataset = dataset.shuffle()
sample_size = 1000
results = classifier(dataset["test"]["review_body"][0:sample_size])
print(f"For {sample_size} samples, the accuracy is {calculate_accuracy(results, dataset)}")

For 1000 samples, the accuracy is 0.74


In [45]:
results = classifier(dataset["test"]["review_body"][0:3500])

RuntimeError: The size of tensor a (582) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
len(results)